### Import Statements

In [1]:
import os
import cv2
import pylab
import dlib
import numpy as np
import pandas as pd
import skimage.io as imageio
import sys
import tensorflow as tf
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
#from keras.utils import multi_gpu_model
from keras.optimizers import SGD, RMSprop
from tensorflow.keras.optimizers import Adam
from keras.layers import Concatenate, Input, concatenate, add, multiply, maximum, LSTM, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline

### GPU Check

In [2]:
gpu_available = tf.config.list_physical_devices('GPU')

In [3]:
gpu_available

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Loading Excel to DataFrame

In [ ]:
#df = pd.read_excel('/Thesis/Datasets/CASME2/CASME2-coding-20190701.xlsx') #index_col=0

In [5]:
df = pd.read_excel('/Thesis/CASME2_aug.xlsx') #index_col=0

In [6]:
df.count()

Subject              510
Filename             510
Unnamed: 2             0
OnsetFrame           510
ApexFrame            510
OffsetFrame          510
Unnamed: 6             0
Action Units         510
Estimated Emotion    510
Number of frames     510
dtype: int64

In [ ]:
df['Estimated Emotion'].unique()

In [7]:
df['Subject'] = df['Subject'].astype(str)
df['Sub_FileName'] = df[['Subject', 'Filename']].apply(lambda x: '_'.join(x), axis=1)
df

,Subject,Filename,Unnamed: 2,OnsetFrame,ApexFrame,OffsetFrame,Unnamed: 6,Action Units,Estimated Emotion,Number of frames,Sub_FileName
0,1,EP02_01f,NaN,46,59,86,NaN,12,happiness,41,1_EP02_01f
1,1,EP03_02,NaN,131,139,161,NaN,18,others,31,1_EP03_02
2,1,EP04_02,NaN,21,54,76,NaN,4,others,56,1_EP04_02
3,1,EP04_03,NaN,31,41,56,NaN,4,others,26,1_EP04_03
4,1,EP04_04,NaN,23,49,66,NaN,4,others,44,1_EP04_04
...,...,...,...,...,...,...,...,...,...,...,...
505,26,EP18_46_aug,NaN,31,46,101,NaN,16,others,71,26_EP18_46_aug
506,26,EP18_47_aug,NaN,6,49,86,NaN,4,disgust,81,26_EP18_47_aug
507,26,EP18_49_aug,NaN,16,54,80,NaN,4,disgust,65,26_EP18_49_aug
508,26,EP18_50_aug,NaN,78,99,161,NaN,4,disgust,84,26_EP18_50_aug


### Combining from  7 classes to 3 classes

In [12]:
positive = df[(df['Estimated Emotion'] == 'happiness' )| (df['Estimated Emotion'] == 'surprise')][['Sub_FileName','ApexFrame','Number of frames']]
print(positive)

       Sub_FileName  ApexFrame  Number of frames
0        1_EP02_01f         59                41
15        2_EP09_01         56               100
18        2_EP11_01         69                91
19        2_EP13_04         51                56
20        2_EP14_01        114                46
..              ...        ...               ...
485  24_EP18_03_aug         76                36
490  25_EP10_10_aug         91                98
491  25_EP12_01_aug        151                56
493  26_EP03_10_aug        149                86
500  26_EP13_02_aug         54                85

[120 rows x 3 columns]


In [9]:
positive = df[(df['Estimated Emotion'] == 'happiness' )| (df['Estimated Emotion'] == 'surprise')][['Sub_FileName','ApexFrame','Number of frames']]
print('Positive :', positive['Sub_FileName'].count())

negative = df[(df['Estimated Emotion'] == 'disgust') | (df['Estimated Emotion'] == 'repression') | (df['Estimated Emotion'] == 'fear') | (df['Estimated Emotion'] == 'sadness')][['Sub_FileName','ApexFrame','Number of frames']]
print('Negative :',negative['Sub_FileName'].count())

neutral = df[df['Estimated Emotion']== 'others'][['Sub_FileName','ApexFrame','Number of frames']]
print('Neutral :',neutral['Sub_FileName'].count())

Positive : 120
Negative : 192
Neutral : 198


### Augmentation function

In [ ]:
def aug_images(subject, vd, img_count, img_flip_lr):
    sub = str(subject)
    video = str(vd) + '_aug'
    main_dir = "/Thesis/CASME2_RAW_selected/"
    sub_path = os.path.join(main_dir, sub) 
    os.makedirs(sub_path, exist_ok=True)
    
    newpath = str(main_dir + sub + '/')
    frame_path = os.path.join(newpath, video)
    frame_path = str(frame_path)
    os.makedirs(frame_path, exist_ok=True)
    
    cv2.imwrite(os.path.join(frame_path , str(img_count)+'.jpg'), img_flip_lr)

### Creating Augmented data in the folders of Training data

In [ ]:
labelpath =  '/Thesis/CASME2_RAW_selected/'
directorylisting = os.listdir(labelpath)
for subject in directorylisting:
    count = 0
    videopath = labelpath + subject + '/'
    Num_videos = os.listdir(videopath)
    for vd in Num_videos:
        framepath = videopath + vd
        framelisting = os.listdir(framepath)
        count = count+1
        img_count = 0
        frames_len = len(framelisting)
        framerange = [x  for x in range(frames_len)]
        for frame in framerange:
            img_count = img_count + 1
            imagepath = framepath + '/' + framelisting[frame]
            image = cv2.imread(imagepath)
            img_flip_lr = cv2.flip(image, 1)
            #aug_images(subject, vd, img_count, img_flip_lr)
        #print('subject : ', subject, '  Videos count : ',count, '  Frames count : ',img_count)

### Pre-processing of frames

In [11]:
training_casme_list = []
onset_offset_frames = 8

In [13]:
num = 0
for lab in (positive['Sub_FileName'], negative['Sub_FileName'], neutral['Sub_FileName']): 
    subDirectory = lab#.tolist() 
    num = num + 1
    i = 0
    count = 0
    for i, sub in tqdm(enumerate(subDirectory)):
        sub = str(sub).split('_')
        labelpath = '/Thesis/Datasets/CASME2/CASME2_RAW_selected/'+'sub'+ sub[0].zfill(2) +'/'
        directorylisting = os.listdir(labelpath)      
        for video in directorylisting:
            if video == str(sub[1]+'_'+sub[2]):
                #frames = []
                total_face_frames = []
                if num == 1:
                    ApexFrame = positive['ApexFrame'].iloc[i]
                    ApexImage = 'img'+ str(ApexFrame) + '.jpg'
                elif num == 2:
                    ApexFrame = negative['ApexFrame'].iloc[i]
                    ApexImage = 'img'+ str(ApexFrame) + '.jpg'
                else:
                    ApexFrame = neutral['ApexFrame'].iloc[i]
                    ApexImage = 'img'+ str(ApexFrame) + '.jpg'
                videopath = labelpath + video
                framelisting = os.listdir(videopath)
                for index, value in enumerate(framelisting):
                    frames = []
                    if str(value) == str(ApexImage):
                        start = index - onset_offset_frames
                        end = index + onset_offset_frames
                        if (len(framelisting) > end) & (start > 0):
                            # print('Subject : ', sub[0].zfill(2), ' video : ', video, ' ApexFrame : ', ApexFrame, '  Apex_Image : ', index, ' start : ', start, ' end : ', end )
                            framerange = [x  for x in range(start, end)]
                            count = count + 1
                            for frame in framerange:
                                imagepath = videopath + "/" + framelisting[frame]
                                image = cv2.imread(imagepath)
                                imageresize = cv2.resize(image, (64, 64), interpolation = cv2.INTER_AREA)
                                grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
                                image_dim = np.expand_dims(grayimage, axis=2)
                                frames.append(image_dim)
                            frames = np.asarray(frames)
                            training_casme_list.append(frames)
    print(count)

120it [00:14,  8.06it/s]


110


192it [00:22,  8.56it/s]


164


198it [00:22,  8.81it/s]

166


In [14]:
len(training_casme_list)

440

In [15]:
training_casme_list[2].shape

(16, 64, 64, 1)

In [16]:
110+164

274

### Creating Target labels

In [17]:
training_casme_list = np.asarray(training_casme_list)
training_casme_samples = len(training_casme_list)

training_casme_labels = np.zeros((training_casme_samples, ), dtype = int)

training_casme_labels[0:110] = 0
training_casme_labels[110:274] = 1
training_casme_labels[274:440] = 2

training_casme_labels = np_utils.to_categorical(training_casme_labels, 3)

In [18]:
training_casme_data = [training_casme_list, training_casme_labels]
(training_casme_set, traininglabels_casme) = (training_casme_data[0], training_casme_data[1])

In [19]:
training_casme_set = training_casme_set.astype('float32')
training_casme_set -= np.mean(training_casme_set)
training_casme_set /= np.max(training_casme_set)

In [20]:
training_casme_set.shape

(440, 16, 64, 64, 1)

#### Save Numpy Arrays

In [21]:
# Save Nump Arrays to save time
np.save('/Thesis/Multi_Scale_MicroExp/Sample_datasets/microexp_casme_images.npy', training_casme_set)
np.save('/Thesis/Multi_Scale_MicroExp/Sample_datasets/microexp_casme_labels.npy', training_casme_labels)

#### Loading Numpy arrays

In [ ]:
training_casme_set = None
training_casme_labels = None
training_casme_set  = np.load('/Thesis/Multi_Scale_MicroExp/Sample_datasets/microexp_casme_images.npy')
training_casme_labels = np.load('/Thesis/Multi_Scale_MicroExp/Sample_datasets/microexp_casme_labels.npy')

training_casme_set.shape

### Model

In [ ]:
model = Sequential()
model.add(Convolution3D(16, (3, 3, 3), input_shape=(16, 64, 64, 1), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])

model.summary()

In [ ]:
# Load pre-trained weights

#model.load_weights('/Thesis/MicroExpSTCNN/Training_dataset/weights_microexpstcnn/weights-improvement-53-0.88.hdf5')

#model.load_weights('/Thesis/MicroExpSTCNN/CASME_SQUARE/weights-improvement-53-0.88.hdf5')

#### Model Compile

In [ ]:
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=10, decay_rate=0.90, staircase=True)

model.compile( loss="categorical_crossentropy", optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics=['accuracy'])

#### Creating checkpoints

In [ ]:
filepath="/Thesis/Multi_Scale_MicroExp/Sample_datasets/weights_microexp/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor="val_accuracy", patience=150)

callbacks_list = [checkpoint, early_stopping]

#### Spliting the dataset into training and validation sets

In [ ]:
train_images, validation_images, train_labels, validation_labels =  train_test_split(training_casme_set, training_casme_labels, test_size=0.2, random_state=4)

#### Training the model

In [ ]:
hist = model.fit(train_images, train_labels, validation_data = (validation_images, validation_labels), callbacks=callbacks_list, batch_size = 8, epochs = 300, shuffle=True)

#### Finding Confusion Matrix

In [ ]:
predictions = model.predict(validation_images)
predictions_labels = np.argmax(predictions, axis=1)
validation_labels = np.argmax(validation_labels, axis=1)
cfm = confusion_matrix(validation_labels, predictions_labels)
print (cfm)